In [41]:
# !pip install chromadb

In [39]:
import sys
import os
from pathlib import Path

project_root = Path.cwd().parent 
if str(project_root) not in sys.path:
    sys.path.insert(0, str(project_root))
from src.predictive_models import (
    load_datasets,
    build_frequency_model,
    predict_frequency_model,
    build_sensationalism_model,
    predict_sensationalism_model,
    build_malicious_account_model,
    predict_malicious_account_model,
    build_naive_realism_model,
    predict_naive_realism_model,
    map_sensationalism_from_counts,
    map_naive_realism_from_sentiment
)

from src.articles import (
    prepare_article
)

import json
from sentence_transformers import SentenceTransformer
from chromadb.config import Settings

# Rag

In [36]:
def chunk_text(text, chunk_size=500, overlap=50):
    chunks, start = [], 0
    while start < len(text):
        end = start + chunk_size
        chunks.append(text[start:end])
        start += chunk_size - overlap
    return chunks

In [37]:
def setup_rag_for_article(article_df):
    text = article_df["statement"].iloc[0]
    chunks = chunk_text(text)
    embedder = SentenceTransformer("all-MiniLM-L6-v2")
    embeddings = embedder.encode(chunks)

    client = chromadb.Client(Settings(anonymized_telemetry=False))
    collection = client.get_or_create_collection(name="articles")
    collection.add(
        ids=[f"chunk_{i}" for i in range(len(chunks))],
        documents=chunks,
        embeddings=embeddings.tolist(),
        metadatas=[{
            "source": article_df["source"].iloc[0],
            "publish_date": article_df["publish_date"].iloc[0]
        } for _ in chunks]
    )
    return collection, embedder

In [38]:
def retrieve_context(query, collection, embedder, n_results=3):
    query_embedding = embedder.encode([query])
    results = collection.query(query_embeddings=query_embedding.tolist(), n_results=n_results)
    retrieved = [doc for doc in results["documents"][0]]
    return "\n\n".join(retrieved)

# Setup

Please ensure you have imported a Gemini API key from AI Studio.
You can do this directly in the Secrets tab on the left.

After doing so, please run the setup cell below.

In [55]:
!pip install -U -q "google"
!pip install -U -q "google.genai"

import os
from google.colab import userdata
from google.colab import drive
os.environ["GEMINI_API_KEY"] = userdata.get("capstone")

# drive.mount("/content/drive")
# # Please ensure that uploaded files are available in the AI Studio folder or change the working folder.
# os.chdir("/content/drive/MyDrive/Google AI Studio")

# Generated Code

In [6]:
article_text = """
Trump’s response to ‘No Kings’ marches only proved the protesters’ point
Analysis by
Stephen Collinson Profile
Stephen Collinson
Oct 19, 2025



President Donald Trump arrives to greet Argentinian President Javier Milei outside the West Wing of the White House in Washington, DC, on October 14.
President Donald Trump arrives to greet Argentinian President Javier Milei outside the West Wing of the White House in Washington, DC, on October 14. Stefani Reynolds/Bloomberg/Getty Images
A president steeped in constitutional lore might have been offended by claims he’s acting as a king.

But Donald Trump and his entourage responded with mockery to weekend “No Kings” protests by millions of Americans — embracing the narrative in a way that explains his growing hubris and belief that he has unchecked power.

The president and Vice President JD Vance this weekend both posted AI memes to social media depicting Trump in a crown. The one Trump shared was a mocked-up vision of the president flying a fighter jet emblazoned with “KING TRUMP” appearing to dump raw sewage on protesters. In Vance’s troll on Bluesky, a site popular with liberals, prominent Democrats knelt as supplicants in a royal court before Trump, the divine and absolute ruler.

It was a clever political trick. MAGA supporters can blast anyone who takes offense to the posts as lacking a sense of humor and being prim. The posts also trivialize protesters’ gravely serious claims that America is witnessing a wannabe autocrat. But they also subtly advance the thesis that Trump is all-powerful and immune from dissent, an attractive proposition to voters who like a strongman.

Attendees sign a banner representing the US Constitution during a "No Kings" protest in Washington, DC, on Saturday.
Attendees sign a banner representing the US Constitution during a "No Kings" protest in Washington, DC, on Saturday. Allison Robbert/AP
Trump’s post is not only juvenile but also betrays striking contempt for tens of millions of Americans he ostensibly leads and for the concept of democratic free speech. He’s far from the first major political figure to express disdain for voters they don’t agree with. Presidents Barack Obama and Joe Biden, as well as 2016 Democratic nominee Hillary Clinton, all had their moments. And a sense that liberal elites disrespected heartland Americans was fundamental to Trump’s rise to power.

Moreover, Democrats seem to have done little self-reflection over whether their aggressive push to implement their progressive values alienated more conservative Americans and contributed to their own defeat in 2024.

But the president’s social media baiting would be easier to laugh off if it weren’t for multiplying actions that reinforce his pretensions to absolute power and the obliteration of constitutional curbs meant to ensure kings don’t again rule America. He’s implying there’s no room for those who don’t support him and showing he’s willing to force them into line.


Whether such leadership is forcing the country inexorably closer to a political and social fracture or a pride-induced fall by Trump and his gang will become clear in the months to come. The stakes are especially high amid a government shutdown the president appears to have no desire to end.

But as Trump’s behavior becomes more imperious and as opposition gathers — from street protests to more universities refusing to bend to his ideological will, and judges temporarily halting his orders for troops to enter US cities — tensions and divisions are rising.

The president shows no sign of changing course. He called Saturday’s mass protests a “joke” and described them as “very small, very ineffective.” The people who took part were “whacked out,” Trump said.


AD
“When you look at those people, those are not representative of the people of our country,” he told reporters aboard Air Force One on Sunday.

Trump’s latest monarchical moves
Trump’s recent track record reinforces perceptions that his denial that he sees himself as a king is yet another of his uncountable untruths.

His stunning commutation Friday night of the seven-year sentence of former Rep. George Santos, who pleaded guilty to fraud charges, was the president’s latest parody of the legal system that he unapologetically wields as a tool to help his friends and hurt his foes. This followed his demands that ex-FBI chief James Comey and New York Attorney General Letitia James be charged with crimes, which were followed by indictments scant weeks later.


AD
Santos on Sunday related to CNN’s Dana Bash how he’d been alerted to his impending release by other prison inmates who heard about it on TV. On “State of the Union,” Santos said, “I’m pretty confident that if President Trump had pardoned Jesus Christ off of the cross, he would have had critics. So that’s just the reality of our country.” But Rep. Nick LaLota, Santos’ former GOP colleague from New York, said of Santos: “He stole millions, defrauded an election, and his crimes (for which he pled guilty) warrant more than a three-month sentence.”

Rep. George Santos is surrounded by journalists as he leaves the US Capitol after his fellow members of Congress voted to expel him from the House of Representatives on December 1, 2023.
Rep. George Santos is surrounded by journalists as he leaves the US Capitol after his fellow members of Congress voted to expel him from the House of Representatives on December 1, 2023. Kevin Dietsch/Getty Images
Pardoning and commutations are one of the few areas where the Constitution suggest presidents do have unaccountable power. But Trump’s treatment of Santos, which was far from his first highly political show of clemency, was an audacious statement that the US legal system’s reputation for blind justice now risks being tainted by made-to-order prosecutions and his regal whims. (Of course, Republicans will respond that Biden pardoned members of his own family, so all is fair game.)

The president’s authoritarian writ is also being extended abroad.

► Defense Secretary Pete Hegseth announced Sunday that the US military conducted a seventh strike against a boat in the Caribbean that the administration claims was manned by narcotics traffickers. “These cartels are the Al Qaeda of the Western Hemisphere, using violence, murder and terrorism to impose their will, threaten our national security and poison our people,” Hegseth said.

► Trump’s promises to get tough with drugs traffickers who have caused great misery inside the US resonated with many voters last year. But the administration’s unilateral designation of traffickers as terrorists and assumption of authority to kill them without due process, while ignoring Congress’ authority over war powers, is arguably illegal and unconstitutional. It places the United States in a category of nations that mock the rule of law and opens the door for others to do the same. “All of these people have been blown up without us knowing their name, without any evidence of a crime,” Republican Sen. Rand Paul said on NBC’s “Meet the Press” on Sunday. “If they want all-out war where we kill anybody and everybody that is in the country of Venezuela or coming out, that has to have a declaration of war,” the Kentucky Republican said. “It’s something that is not pretty, very expensive, and I’m not in favor of declaring war on Venezuela, but Congress should vote. The president shouldn’t do this by himself.”

► Trump last week detailed his decision to authorize covert CIA operations in Venezuela, prompting debate over a possible regime-change strategy against the brutal rule of President Nicolás Maduro. On Sunday, Trump stoked speculation of further US military action in the Western Hemisphere, warning that if Colombian President Gustavo Petro didn’t “close up these killing fields” where drugs are produced, “the United States will close them up for him and it won’t be done nicely.”

Curtailing drugs trafficking might be a laudable goal, but Trump’s methods threaten to bolster an imperial presidency and change America’s global reputation. His administration’s refusal to produce evidence of its claims or to try to build support for a potential military campaign is another example of contempt for the democratic process — one that is fueled by the genuflection of most Republicans in Congress.


AD
The saga is unfolding as Hegseth’s Defense Department has kicked out journalists who refused to sign the administration’s draconian new press regulations. The civilian leadership of Pentagon looks scared of scrutiny — a position that risks eroding public support for the military. That possibility is only reinforced by the administration’s determination to deploy troops on American soil.

Protesters challenge the administration’s dismissal of demonstrations
Trump aides tried to diminish and defame the “No Kings” protests that drew millions of people to more than 2,700 events in 50 states and were largely peaceful. White House press secretary Karoline Leavitt claimed the Democratic Party’s constituency was made up of “Hamas terrorists, illegal aliens, and violent criminals.” House Speaker Mike Johnson branded them “hate America” rallies made up of the “pro-Hamas wing” and “the Antifa people.”

There is a seam of the Democratic base nurtured by left-wing protest movements that many centrist Americans regard as extreme. But impressions gathered around the country by CNN reporters also indicated that many people with more moderate views also took part in protests that felt like block parties to register concerns about a president many of them see as un-American. Demonstrators dressed as frogs, chickens and Statues of Liberty mocked the unhinged rhetoric of the administration about the protests just as effectively as the president’s vulgar social media posts mocked them.

People gather along a waterfront park during a "No Kings" protest in Portland, Oregon, on Saturday.
People gather along a waterfront park during a "No Kings" protest in Portland, Oregon, on Saturday. Jenny Kane/AP
“We have millions of people turning out today,” Colleen Connell, executive director of the Illinois chapter of the American Civil Liberties Union, told CNN’s Fredricka Whitfield on Saturday. “There are clearly more of us than there are of those who want an autocracy or a dictatorship. And so the message is pretty clear that we, the people, you know, own this country and that we are going to respect the rule of law and we are going to stand up and we are going to protect our democracy with peaceful protest and peaceful dissent.”

In time, Saturday’s protests may come to be seen not just as a reaction to Trump’s first nine months in power but as a harbinger of greater resistance to come.

At the least, they show that while many of Trump’s supporters back tough-guy policies like expanding Immigrations and Custom Enforcement raids and extrajudicial executions of claimed drugs traffickers on the high seas, many other Americans don’t.

And while warnings of encroaching authoritarianism may initially have seemed overblown, growing numbers of Americans seem to think that sitting on the sidelines is untenable. This follows a warning by Obama on Marc Maron’s final podcast that it wouldn’t be a huge sacrifice for more Americans to make their views known a year before midterm elections. “We’re not at the stage where you have to be like Nelson Mandela and be in a 10-by-12 jail cell for 27 years and break rocks,” the former president said.

Organizers claimed 7 million protesters came out at the weekend. If that’s true, that means nearly 10% of the 75 million people who voted for Kamala Harris last November left their homes to join a protest.

A person dressed as former US President George Washington displays a sign during a "No Kings" protest in Boston on Saturday.
A person dressed as former US President George Washington displays a sign during a "No Kings" protest in Boston on Saturday. Brian Snyder/Reuters
A protester dressed in a Statue of Liberty costume takes part in a "No Kings" protest in New York on Saturday.
A protester dressed in a Statue of Liberty costume takes part in a "No Kings" protest in New York on Saturday. Olga Fedorova/AP
Retired government worker Peggy Cole of Flint, Michigan, told CNN’s Veronica Stracqualursi she traveled to Washington for a protest because it was a “scary time.”

“It seems to me, (Trump is) taking our government, our democracy, and dismantling it piece by piece, slowly, but surely, if we sit by and don’t do anything about it,” Cole said.

Asked for comment about the protests, White House spokesperson Abigail Jackson wrote back: “Who cares?”

That’s a response that breathes contempt for the values protesters claim to be protecting. But it also reveals a White House that believes its increasingly monarchical leader has no need to listen to the aspirations of those he regards as below him.

In some ways, Trump is proving the protesters’ point.
"""

In [35]:
train_path = "../data/train_set.csv"
val_path = "../data/val_set.csv"
test_path = "../data/test_set.csv"
df_train, df_val, df_test = load_datasets(
    train_path, val_path, test_path
)

train_freq = df_train.copy()
val_freq   = df_val.copy()
test_freq  = df_test.copy()

cnn_df = prepare_article(article_text)

In [11]:
train_freq = df_train.copy()

model, tfidf, count_vec, token_dict, buzzwords, label_encoder = build_frequency_model(train_freq)

freq_pred = predict_frequency_model(cnn_df, model, tfidf, count_vec, token_dict, buzzwords, label_encoder)

In [12]:
train_sens = df_train.copy()

sens_model, sens_numeric_features = build_sensationalism_model(train_sens)

sens_pred = predict_sensationalism_model(cnn_df, sens_model, sens_numeric_features)

In [15]:
train_mal = df_train.copy()

mal_model, mal_tfidf, mal_le = build_malicious_account_model(train_mal)

ma_pred = predict_malicious_account_model(cnn_df, mal_model, mal_tfidf, mal_le)



In [14]:
train_nr = df_train.copy()

naive_model, naive_numeric_features = build_naive_realism_model(train_nr)

nr_pred = predict_naive_realism_model(cnn_df, naive_model, naive_numeric_features)


In [16]:
prompt = f"""
You are an expert in misinformation and disinformation detection.
Your task is to analyze the given article and score how strongly it exhibits each factuality factor.
---

### Factuality Factors:
1. **Frequency Heuristic**
  - *Repetition Analysis*: Observe how often a claim or narrative is echoed across the text or across references.
  - *Origin Tracing*: Determine whether frequently repeated information is traced to a credible or questionable source.
  - *Evidence Verification*: Evaluate if the text implies truth merely due to repetition or popularity of a claim.
  - **Scoring:** 0 = none/minimal repetition; 1 = moderate repetition or common-belief phrasing; 2 = heavy repetition or appeal to consensus.

2. **Malicious Account**
  - *Account Analysis*: If the text references or cites accounts, consider whether their creation dates or activity patterns suggest inauthentic behavior.
  - *Interaction Patterns*: Evaluate if the content originates from or interacts with accounts resembling coordinated or bot-like behavior.
  - *Content Review*: Assess if the account or source repeatedly spreads false or harmful information.
  - **Scoring:** 0 = credible source; 1 = slightly suspicious or biased source; 2 = clearly deceptive, coordinated, or malicious source.

3. **Sensationalism**
  - *Language Intensity*: Examine the text for overly dramatic or exaggerated claims.
  - *Tone Comparison*: Compare emotional tone of headlines versus main content.
  - *Shock vs. Substance*: Determine whether the article prioritizes shock value over factual reporting.
  - **Scoring:** 0 = neutral/objective; 1 = mildly emotional or dramatic; 2 = highly sensationalized

4. **Naive Realism**
  - *Perspective Analysis*: Evaluate whether the content presents its view as the only correct one.
  - *Dissenting View Checks*: Analyze whether differing views are acknowledged or dismissed.
  - *Isolation Analysis*: Determine whether the article attempts to isolate readers from alternative perspectives.
  - **Scoring:** 0 = balanced and nuanced; 1 = somewhat one-sided; 2 = fully dogmatic.

You are also given reference model scores from predictive models.
Treat these as informative context, not ground truth, and reason independently.

### Reference Model Scores
- Frequency Heuristic: {freq_pred}
- Malicious Account: {ma_pred}
- Sensationalism: {sens_pred}
- Naive Realism: {nr_pred}

### Article to Evaluate:
{article_text}

### Instructions:
1. Think step-by-step about the article’s tone, evidence, framing, and intent.
2. Identify linguistic cues that signal bias, repetition, exaggeration, or malicious intent.
3. Provide a numeric score and a justification for why that score was chosen.
4. Return **only** a valid JSON object containing the score and reasoning for each of the four factors.

### Output Format:
{{
    "frequency_heuristic": {{
        "score": 0|1|2,
        "reasoning": "Explanation"
  }},
    "malicious_account": {{
        "score": 0|1|2,
        "reasoning": "Explanation"
  }},
    "sensationalism": {{
        "score": 0|1|2,
        "reasoning": "Explanation"
  }},
    "naive_realism": {{
        "score": 0|1|2,
        "reasoning": "Explanation"
  }}
}}
"""

In [20]:
# To run this code you need to install the following dependencies:
# pip install google-genai

import base64
import os
from google import genai
from google.genai import types


In [26]:
client = genai.Client(
        api_key=os.environ.get("GEMINI_API_KEY"),
    )
def generate(prompt):

    model = "gemini-2.5-pro"
    contents = [
        types.Content(
            role="user",
            parts=[
                types.Part.from_text(text=prompt),
            ],
        ),
    ]
    tools = [
        types.Tool(googleSearch=types.GoogleSearch(
        )),
    ]
    generate_content_config = types.GenerateContentConfig(
        thinking_config = types.ThinkingConfig(
            thinking_budget=-1,
        ),
        tools=tools,
    )

    for chunk in client.models.generate_content_stream(
        model=model,
        contents=contents,
        config=generate_content_config,
    ):
        print(chunk.text, end="")

if __name__ == "__main__":
    generate(prompt)


```json
{
    "frequency_heuristic": {
        "score": 2,
        "reasoning": "The article heavily repeats the central narrative that President Trump is acting like a king or an autocrat. This idea is introduced in the headline ('proved the protesters’ point') and is reiterated throughout the piece using various synonyms and phrases such as 'wannabe autocrat,' 'unchecked power,' 'all-powerful,' 'pretensions to absolute power,' 'monarchical moves,' 'authoritarian writ,' and 'imperial presidency.' The entire analysis is built around this recurring theme, using each new piece of evidence to reinforce the same core claim, which constitutes a heavy repetition of a single narrative."
    },
    "malicious_account": {
        "score": 0,
        "reasoning": "The article is an analysis piece written by a named journalist, Stephen Collinson, and attributes information to credible sources such as other CNN reporters, public figures (Sen. Rand Paul, Rep. Nick LaLota), and verifiable social med

# Different Prompting

In [27]:
def run_prompt(prompt_text, json_mode=False):
    contents = [
        types.Content(
            role="user",
            parts=[types.Part(text=prompt_text)],
        ),
    ]

    config_kwargs = {
        "thinking_config": types.ThinkingConfig(
            thinking_budget=2048,
        ),
    }
    if json_mode:
        config_kwargs["response_mime_type"] = "application/json"

    generate_content_config = types.GenerateContentConfig(**config_kwargs)

    response = client.models.generate_content(
        model="gemini-2.5-pro",
        contents=contents,
        config=generate_content_config,
    )

    print("RAW RESPONSE:\n")
    print(response.text)

    parsed = None
    if json_mode:
        try:
            parsed = json.loads(response.text)
            print("\nParsed JSON keys:", parsed.keys())
        except json.JSONDecodeError as e:
            print("\nJSON parse failed:", e)

    return response.text, parsed

In [28]:
prompt_baseline = f"""
You are an expert in misinformation and disinformation detection.

Your task is to read the article below and score how strongly it exhibits each of the following factuality factors:

1. Frequency Heuristic (0–2)
   - 0 = little or no repetition of a claim as a shortcut for truth.
   - 1 = some repetition or “people say…” framing.
   - 2 = strong reliance on repetition or popularity to imply truth.

2. Malicious Account (0–2)
   - 0 = credible, professional, or clearly attributed sources.
   - 1 = somewhat suspicious or biased sources, but not clearly malicious.
   - 2 = clearly deceptive, anonymous, or coordinated behavior.

3. Sensationalism (0–2)
   - 0 = neutral, measured tone.
   - 1 = mildly emotional or dramatic, but still grounded.
   - 2 = highly sensationalized, shock-driven language.

4. Naive Realism (0–2)
   - 0 = balanced, acknowledges alternative views.
   - 1 = somewhat one-sided, but not totally closed off.
   - 2 = dogmatic, treats its view as the only rational one.

INSTRUCTIONS:
- Ignore any external models or tools.
- Just use your own reading of the article.
- For each factor:
  - Give a numeric score from 0–2.
  - Provide 2–3 sentences explaining why.

ARTICLE:
{article_text}

OUTPUT FORMAT (plain text is fine for this pass):
- Frequency Heuristic: <score> – <short explanation>
- Malicious Account: <score> – <short explanation>
- Sensationalism: <score> – <short explanation>
- Naive Realism: <score> – <short explanation>
"""

raw_baseline, _ = run_prompt(prompt_baseline, json_mode=False)


RAW RESPONSE:

- **Frequency Heuristic: 0** – The article builds its argument by citing a variety of different events and statements rather than by repeating a single claim. It presents multiple distinct examples—social media posts, a presidential commutation, military actions, and public comments—to support its central thesis. It does not rely on repetition as a shortcut to establish truth.

- **Malicious Account: 0** – The sources are credible and clearly attributed to named individuals, established news outlets (CNN, NBC), and major photo agencies (AP, Reuters, Getty). The piece is published under a byline, indicating clear authorship and accountability. There is no evidence of anonymous, deceptive, or coordinated inauthentic behavior.

- **Sensationalism: 1** – The tone is not neutral, employing emotionally charged and interpretive language such as "wannabe autocrat," "striking contempt," and "imperious." However, this language is used within the context of a clearly labeled "Analy

In [29]:
engineering_prompts = []

In [30]:
# 2.1 – Adds explicit factor headings
engineering_prompts.append(f"""
You are analyzing the same article as before.

For each of the four factuality factors:
1. Frequency Heuristic
2. Malicious Account
3. Sensationalism
4. Naive Realism

Give:
- A short description of how the article behaves along that dimension.
- A score from 0–2.

Be explicit, like:

Factor: Frequency Heuristic
Score: X
Explanation: ...

ARTICLE:
{article_text}
""")

In [31]:
raw_eng1, parsed_eng1 = run_prompt(engineering_prompts[0], json_mode=False)

RAW RESPONSE:

Factor: Frequency Heuristic
Score: 2
Explanation: The article relentlessly repeats its central thesis: that Donald Trump is behaving like a king or an autocrat. This idea is introduced in the headline ("proved the protesters' point") and is reinforced throughout the piece by using words like "king," "monarchical," "hubris," "unchecked power," "autocrat," "absolute power," and "imperious." Every action described, from social media posts to foreign policy, is framed as further evidence of this single theme, and the article concludes by restating the initial premise. This high level of repetition is designed to make the central claim feel more established and true.

Factor: Malicious Account
Score: 0
Explanation: The article is not from a malicious account. It is explicitly labeled as "Analysis" by a named author (Stephen Collinson) from a well-known, established news organization (CNN). While the content is highly critical and presents a strong viewpoint, it is transparent

In [ ]:
# 2.2 – Require explicit rubric reference
engineering_prompts.append(f"""
Using the definitions and scale below, score the article.

Frequency Heuristic (0–2): repetition and appeal to popularity.
Malicious Account (0–2): malicious or coordinated accounts as source.
Sensationalism (0–2): exaggerated or emotional language.
Naive Realism (0–2): presents its view as the only valid one.

For each factor:
- Mention specific phrases or sections of the article when possible.
- Then give a score.

ARTICLE:
{article_text}
""")

In [ ]:
raw_eng2, parsed_eng2 = run_prompt(engineering_prompts[1], json_mode=False)

RAW RESPONSE:

Based on the definitions provided, here is a scoring of the article for each factor.

### **Frequency Heuristic (0–2)**
*   **Analysis:** The article heavily relies on repetition of a central theme and an appeal to the popularity of that theme.
    *   **Repetition:** The idea that Trump is a "king," "monarch," or "autocrat" is repeated consistently throughout the text. Specific phrases include: "acting as a king," "No Kings’ protests," "wannabe autocrat," "divine and absolute ruler," "pretensions to absolute power," "behavior becomes more imperious," "latest monarchical moves," "regal whims," "authoritarian writ," and "imperial presidency." This repetition serves to reinforce the article's central argument.
    *   **Appeal to Popularity:** The article repeatedly cites the large number of protesters as evidence for the validity of its claims. Phrases include: "protests by millions of Americans," "millions of people turning out today," "2,700 events in 50 states," and "O

In [ ]:
# 2.3 – Ask for bullet point evidence
engineering_prompts.append(f"""
For each factor (Frequency Heuristic, Malicious Account, Sensationalism, Naive Realism):

1. List 2–4 bullet points of evidence from the article (paraphrased).
2. Then give a final score 0–2 and a one-sentence reason.

ARTICLE:
{article_text}
""")

In [ ]:
raw_eng3, parsed_eng3 = run_prompt(engineering_prompts[2], json_mode=False)

RAW RESPONSE:

Here are the analyses for each factor based on the provided article.

### **Frequency Heuristic**

*   **Evidence:**
    *   The article repeatedly uses words like “king,” “monarchical,” “autocrat,” and “imperial” to describe the president and his actions.
    *   The central framing of the piece is the “No Kings” protest, which is mentioned at the beginning, middle, and end to reinforce the theme of monarchical behavior.
    *   The concept of Trump having or seeking “unchecked power,” “absolute power,” or being “all-powerful” is mentioned multiple times throughout the text.
*   **Score & Reason:**
    *   **Score: 2**
    *   **Reason:** The article consistently repeats the "king" and "autocrat" theme to frame every action, making this characteristic seem pervasive and central to the political situation.

### **Malicious Account**

*   **Evidence:**
    *   The president’s social media account shared an AI meme depicting him as "KING TRUMP" dumping raw sewage on protes

In [ ]:
# 2.4 – Ask for low / medium / high + numeric mapping
engineering_prompts.append(f"""
Evaluate the article along four factors.

For each factor:
- First classify as "low", "medium", or "high".
- Then map that to 0, 1, or 2.
- Explain why.

Factors:
- Frequency Heuristic
- Malicious Account
- Sensationalism
- Naive Realism

ARTICLE:
{article_text}
""")

In [ ]:
raw_eng4, parsed_eng4 = run_prompt(engineering_prompts[3], json_mode=False)

RAW RESPONSE:

Based on the article provided, here is an evaluation along the four requested factors.

### **Frequency Heuristic**

*   **Classification:** High
*   **Map:** 2
*   **Explanation:** The article relentlessly repeats the central theme that Donald Trump is acting like a king or an autocrat. This idea is introduced in the first sentence and is used as the framing device for every subsequent point. Words and phrases like “king,” “monarchical,” “autocrat,” “unchecked power,” “absolute ruler,” “imperious,” and “authoritarian” are used repeatedly throughout the text. By framing every action—from social media posts to military strikes and presidential pardons—as further proof of this single "Trump as king" thesis, the article makes the concept feel pervasive and highly significant, which is the core effect of the frequency heuristic.

### **Malicious Account**

*   **Classification:** Low
*   **Map:** 0
*   **Explanation:** This factor assesses whether the source of the informati

In [ ]:
# 2.5 – Introduce uncertainty
engineering_prompts.append(f"""
For each factuality factor, provide:
- A 0–2 score.
- A 0–1 confidence value (how certain you are).
- 2–3 sentences explaining the decision.

Factors: Frequency Heuristic, Malicious Account, Sensationalism, Naive Realism.

ARTICLE:
{article_text}
""")

In [ ]:
raw_eng5, parsed_eng5 = run_prompt(engineering_prompts[4], json_mode=False)

RAW RESPONSE:

Here is an analysis of the article for each factuality factor.

### **Frequency Heuristic**

*   **Score:** 2/2
*   **Confidence:** 1.0
*   **Explanation:** The article constructs its fictional narrative by extrapolating from real, frequently repeated criticisms and concerns about Donald Trump. It heavily relies on the audience's existing familiarity with themes like his supposed authoritarian tendencies, use of pardons for allies, and "strongman" rhetoric to make its speculative scenario feel plausible and resonant. This leveraging of established, high-frequency narratives is a clear and effective use of the heuristic to build a persuasive, albeit fictional, argument.

### **Malicious Account**

*   **Score:** 0/2
*   **Confidence:** 0.9
*   **Explanation:** The article is a piece of speculative political commentary, not a factual report from a deceptive source. While it is highly critical, its use of easily verifiable counter-factuals (e.g., JD Vance as Vice President,

In [ ]:
# 2.6 – Ask to separate "language" vs "structure"
engineering_prompts.append(f"""
For each factor, structure your answer like this:

- Language cues: ...
- Structural cues: ...
- Score (0–2): ...

Factors:
- Frequency Heuristic
- Malicious Account
- Sensationalism
- Naive Realism

ARTICLE:
{article_text}
""")

In [ ]:
raw_eng6, parsed_eng6 = run_prompt(engineering_prompts[5], json_mode=False)

RAW RESPONSE:

Here is the analysis of the article for each factor.

### Frequency Heuristic

-   **Language cues:** The article uses words and phrases that suggest a repeated and escalating pattern of behavior, such as "growing hubris," "multiplying actions," "not only... but also," "his latest monarchical moves," "far from his first," and "another example of contempt." This language reinforces the idea that the described actions are not isolated incidents but part of a continuous and worsening trend.
-   **Structural cues:** The article is structured as a list of grievances or proofs of its central thesis. It moves from one example to the next (social media posts, the Santos commutation, demands for indictments, military strikes, press regulations) to create an overwhelming impression of a consistent pattern of authoritarian behavior. By stacking these examples, it makes the "kingly" behavior seem constant and pervasive.
-   **Score (0–2):** 2

### Malicious Account

-   **Language c

In [ ]:
# 2.7 – Ask to list specific quoted snippets
engineering_prompts.append(f"""
For each factor, do:

1. List up to 3 short snippets (quoted or paraphrased) that support your judgment.
2. Then give a 0–2 score and explain.

Factors:
- Frequency Heuristic
- Malicious Account
- Sensationalism
- Naive Realism

ARTICLE:
{article_text}
""")

In [ ]:
raw_eng7, parsed_eng7 = run_prompt(engineering_prompts[6], json_mode=False)

RAW RESPONSE:

Based on the article provided, here is an analysis of each factor.

### Frequency Heuristic

1.  **Snippets:**
    *   The article repeatedly uses words and concepts related to royalty and autocracy to describe Trump: "acting as a king," "No Kings protests," "KING TRUMP," "divine and absolute ruler," "wannabe autocrat," "monarchical moves," "regal whims," "imperial presidency," and "increasingly monarchical leader."
    *   The idea of Trump having "unchecked power" or "absolute power" is mentioned at the beginning ("belief that he has unchecked power") and reinforced later ("pretensions to absolute power").
    *   The administration's "contempt" for dissent and democratic processes is stated multiple times: "betrays striking contempt for tens of millions of Americans," "contempt for the democratic process," and "a response that breathes contempt for the values protesters claim to be protecting."

2.  **Score and Explanation:**
    *   **Score: 2**
    *   The article r

In [ ]:
# 2.8 – Ask to explicitly mention ambiguity
engineering_prompts.append(f"""
For each factor:

1. Briefly describe how the article behaves on that factor.
2. Mention any ambiguities or edge cases.
3. Give a final 0–2 score.

Factors:
- Frequency Heuristic
- Malicious Account
- Sensationalism
- Naive Realism

ARTICLE:
{article_text}
""")

In [ ]:
raw_eng8, parsed_eng8 = run_prompt(engineering_prompts[7], json_mode=False)

RAW RESPONSE:

Here is the analysis of the article for each factor.

### **Frequency Heuristic**

1.  **Description:** The article makes extensive and repeated use of a central theme: Donald Trump's "monarchical" or "king-like" behavior. This concept is the article's core thesis and is reinforced in nearly every paragraph. Phrases like "acting as a king," "No Kings protests," "unchecked power," "divine and absolute ruler," "wannabe autocrat," "pretensions to absolute power," "imperious," "monarchical moves," and "regal whims" are used relentlessly. This high frequency of related terms is designed to make the central claim feel more significant, pervasive, and self-evident to the reader.
2.  **Ambiguities:** The primary ambiguity is whether this is a manipulative heuristic or simply strong, thematic writing for an opinion piece. An analysis article is expected to have a central thesis and bring evidence to support it. However, the sheer density and repetition of the "king" metaphor go b

In [ ]:
# 2.9 – Ask for compact tabular summary in text
engineering_prompts.append(f"""
Summarize your assessment as a small table in plain text, with columns:

Factor | Score (0–2) | One-sentence justification

Then below the table, optionally add a short paragraph explaining the overall framing.

Factors: Frequency Heuristic, Malicious Account, Sensationalism, Naive Realism.

ARTICLE:
{article_text}
""")

In [ ]:
raw_eng9, parsed_eng9 = run_prompt(engineering_prompts[8], json_mode=False)

RAW RESPONSE:

| Factor | Score (0–2) | One-sentence justification |
| :--- | :--- | :--- |
| Frequency Heuristic | 2 | The article lists a rapid succession of different events to create an overwhelming impression of a single, accelerating trend toward authoritarianism. |
| Malicious Account | 1 | While not presenting verifiable falsehoods due to its speculative nature, the piece mixes real figures with a fictional timeline to create a highly damaging and potentially misleading narrative. |
| Sensationalism | 2 | The text uses loaded and dramatic language ("wannabe autocrat," "obliteration of constitutional curbs," "regal whims") to frame political actions in the most alarming way possible. |
| Naive Realism | 2 | The article presents its critical view of the administration as the only rational perspective, framing opposing views and supporters as simply contemptuous, power-hungry, or unreasonable. |

The article is a piece of speculative political analysis, framing a hypothetical near

In [ ]:
# 2.10 – First time requiring JSON
engineering_prompts.append(f"""
You are now required to output a machine-readable JSON object.

Read the article and produce:

{{
  "frequency_heuristic": {{
    "score": 0,
    "reasoning": "..."
  }},
  "malicious_account": {{
    "score": 0,
    "reasoning": "..."
  }},
  "sensationalism": {{
    "score": 0,
    "reasoning": "..."
  }},
  "naive_realism": {{
    "score": 0,
    "reasoning": "..."
  }}
}}

Only return JSON, no extra text.

ARTICLE:
{article_text}
""")

In [ ]:
raw_eng10, parsed_eng10 = run_prompt(engineering_prompts[9], json_mode=True)

RAW RESPONSE:

{
  "frequency_heuristic": {
    "score": 0,
    "reasoning": "The article is a singular piece of political analysis and commentary. It does not argue its points by referencing the volume or frequency of reporting on the subject matter."
  },
  "malicious_account": {
    "score": 0,
    "reasoning": "The article, while presenting a speculative and highly critical political narrative, appears to be a piece of commentary or analysis from a news organization. There are no direct indicators, such as signs of automation, impersonation, or promotion of harmful disinformation, that would suggest it originates from a malicious account."
  },
  "sensationalism": {
    "score": 9,
    "reasoning": "The article consistently employs strong, emotionally charged, and dramatic language to frame the political situation as a crisis. Phrases like 'unchecked power,' 'wannabe autocrat,' 'obliteration of constitutional curbs,' 'pride-induced fall,' 'authoritarian writ,' and 'unhinged rhetori

In [ ]:
# 2.11 – Same JSON, but with explicit internal thinking
engineering_prompts.append(f"""
Internally, think step-by-step about each factor, but do NOT show your chain-of-thought.
Externally, output ONLY the JSON object in the exact schema below:

{{
  "frequency_heuristic": {{
    "score": 0,
    "reasoning": "short explanation"
  }},
  "malicious_account": {{
    "score": 0,
    "reasoning": "short explanation"
  }},
  "sensationalism": {{
    "score": 0,
    "reasoning": "short explanation"
  }},
  "naive_realism": {{
    "score": 0,
    "reasoning": "short explanation"
  }}
}}

ARTICLE:
{article_text}
""")

In [ ]:
raw_eng11, parsed_eng11 = run_prompt(engineering_prompts[10], json_mode=True)

RAW RESPONSE:

{
  "frequency_heuristic": {
    "score": 0,
    "reasoning": "The article is a single, detailed piece of political commentary. It does not show characteristics of being part of a high-frequency, automated, or coordinated campaign. It is a long-form article with specific arguments and citations."
  },
  "malicious_account": {
    "score": 8,
    "reasoning": "The article presents a fictional, future political scenario as if it were a factual news report. It uses real political figures (Donald Trump, JD Vance) but describes events that have not occurred (e.g., a post-2024 Trump presidency, specific protests, and military actions). This framing is intentionally deceptive, blurring the line between speculative commentary and fabricated news, which could mislead readers into believing these hypothetical events are real."
  },
  "sensationalism": {
    "score": 8,
    "reasoning": "The article employs highly sensational and emotionally charged language throughout to construct

In [ ]:
# 2.12 – JSON + hint that model scores exist
engineering_prompts.append(f"""
We will eventually compare your scores to some model outputs,
but for THIS prompt you must ignore any model scores and only use your own judgment.

Internally, think step-by-step. Externally, output ONLY this JSON:

{{
  "frequency_heuristic": {{
    "score": 0,
    "reasoning": "short explanation"
  }},
  "malicious_account": {{
    "score": 0,
    "reasoning": "short explanation"
  }},
  "sensationalism": {{
    "score": 0,
    "reasoning": "short explanation"
  }},
  "naive_realism": {{
    "score": 0,
    "reasoning": "short explanation"
  }}
}}

ARTICLE:
{article_text}
""")

In [ ]:
raw_eng12, parsed_eng12 = run_prompt(engineering_prompts[11], json_mode=True)

RAW RESPONSE:

{
  "frequency_heuristic": {
    "score": 0,
    "reasoning": "The article is a single news analysis piece; the frequency heuristic, which relates to repeated exposure to information, does not apply to a standalone document."
  },
  "malicious_account": {
    "score": 0,
    "reasoning": "The article is from CNN, a mainstream news organization. While it has a clear editorial perspective, it is not a malicious account known for spreading disinformation."
  },
  "sensationalism": {
    "score": 8,
    "reasoning": "The article uses highly emotional and evocative language, such as 'wannabe autocrat,' 'obliteration of constitutional curbs,' and 'regal whims,' to create a sense of alarm and crisis."
  },
  "naive_realism": {
    "score": 7,
    "reasoning": "The article presents its critical interpretation of events as the only objective reality, framing opposing viewpoints as juvenile, contemptuous, or a political trick, without seriously engaging with them."
  }
}

Parsed J

# In Context Learning




In [32]:
prompt = f"""
You are an expert in misinformation and disinformation detection.
Your task is to analyze the given article and score how strongly it exhibits each factuality factor.
---

### Factuality Factors:
1. **Frequency Heuristic**
  - *Repetition Analysis*: Observe how often a claim or narrative is echoed across the text or across references.
  - *Origin Tracing*: Determine whether frequently repeated information is traced to a credible or questionable source.
  - *Evidence Verification*: Evaluate if the text implies truth merely due to repetition or popularity of a claim.
  - **Scoring:** 0 = none/minimal repetition; 1 = moderate repetition or common-belief phrasing; 2 = heavy repetition or appeal to consensus.

2. **Malicious Account**
  - *Account Analysis*: If the text references or cites accounts, consider whether their creation dates or activity patterns suggest inauthentic behavior.
  - *Interaction Patterns*: Evaluate if the content originates from or interacts with accounts resembling coordinated or bot-like behavior.
  - *Content Review*: Assess if the account or source repeatedly spreads false or harmful information.
  - **Scoring:** 0 = credible source; 1 = slightly suspicious or biased source; 2 = clearly deceptive, coordinated, or malicious source.

3. **Sensationalism**
  - *Language Intensity*: Examine the text for overly dramatic or exaggerated claims.
  - *Tone Comparison*: Compare emotional tone of headlines versus main content.
  - *Shock vs. Substance*: Determine whether the article prioritizes shock value over factual reporting.
  - **Scoring:** 0 = neutral/objective; 1 = mildly emotional or dramatic; 2 = highly sensationalized

4. **Naive Realism**
  - *Perspective Analysis*: Evaluate whether the content presents its view as the only correct one.
  - *Dissenting View Checks*: Analyze whether differing views are acknowledged or dismissed.
  - *Isolation Analysis*: Determine whether the article attempts to isolate readers from alternative perspectives.
  - **Scoring:** 0 = balanced and nuanced; 1 = somewhat one-sided; 2 = fully dogmatic.

### Article to Evaluate:
{article_text}

### Instructions:
1. Think step-by-step about the article’s tone, evidence, framing, and intent.
2. Identify linguistic cues that signal bias, repetition, exaggeration, or malicious intent.
3. Provide a numeric score and a justification for why that score was chosen.
4. Return **only** a valid JSON object containing the score and reasoning for each of the four factors.

### Output Format:
{{
    "frequency_heuristic": {{
        "score": 0|1|2,
        "reasoning": "Explanation"
  }},
    "malicious_account": {{
        "score": 0|1|2,
        "reasoning": "Explanation"
  }},
    "sensationalism": {{
        "score": 0|1|2,
        "reasoning": "Explanation"
  }},
    "naive_realism": {{
        "score": 0|1|2,
        "reasoning": "Explanation"
  }}
}}
"""

In [33]:
generate(prompt)

```json
{
    "frequency_heuristic": {
        "score": 1,
        "reasoning": "The article repeats the central claim that Trump is acting like a king and that his actions reinforce this narrative. This theme is echoed throughout the text, referencing his social media posts, commutation of George Santos, and foreign policy decisions. While the claim is repeated, it is linked to different specific events, which prevents it from being scored as heavy repetition. The phrasing appeals to a common belief among the article's likely target audience."
  },
    "malicious_account": {
        "score": 1,
        "reasoning": "The article is written by Stephen Collinson, a political analyst for CNN. CNN is a major news organization, but it is also frequently accused of having a left-leaning bias. Media bias watchdogs rate CNN's website as 'Skews Left' or 'Lean Left'. The author's previous work has been described as critical of conservatives. The article presents an 'Analysis' which inherently in

In [ ]:
prompt_icl = f"""
You are an expert in misinformation and disinformation detection, scoring, and ranking.
Your task is to analyze the given article and score how strongly it exhibits each factuality factor.
---

### Factuality Factors:
1. **Frequency Heuristic**
  - *Repetition Analysis*: Observe how often a claim or narrative is echoed across the text or across references.
  - *Origin Tracing*: Determine whether frequently repeated information is traced to a credible or questionable source.
  - *Evidence Verification*: Evaluate if the text implies truth merely due to repetition or popularity of a claim.
  - **Scoring:** 0 = none/minimal repetition; 1 = moderate repetition or common-belief phrasing; 2 = heavy repetition or appeal to consensus.

2. **Malicious Account**
  - *Account Analysis*: If the text references or cites accounts, consider whether their creation dates or activity patterns suggest inauthentic behavior.
  - *Interaction Patterns*: Evaluate if the content originates from or interacts with accounts resembling coordinated or bot-like behavior.
  - *Content Review*: Assess if the account or source repeatedly spreads false or harmful information.
  - **Scoring:** 0 = credible source; 1 = slightly suspicious or biased source; 2 = clearly deceptive, coordinated, or malicious source.

3. **Sensationalism**
  - *Language Intensity*: Examine the text for overly dramatic or exaggerated claims.
  - *Tone Comparison*: Compare emotional tone of headlines versus main content.
  - *Shock vs. Substance*: Determine whether the article prioritizes shock value over factual reporting.
  - **Scoring:** 0 = neutral/objective; 1 = mildly emotional or dramatic; 2 = highly sensationalized

4. **Naive Realism**
  - *Perspective Analysis*: Evaluate whether the content presents its view as the only correct one.
  - *Dissenting View Checks*: Analyze whether differing views are acknowledged or dismissed.
  - *Isolation Analysis*: Determine whether the article attempts to isolate readers from alternative perspectives.
  - **Scoring:** 0 = balanced and nuanced; 1 = somewhat one-sided; 2 = fully dogmatic.

### Examples

**Example 1:**
Article: The Federal Reserve announced a 0.25% interest rate increase following its quarterly meeting. Economists had predicted the move based on recent inflation data.
Frequency Heuristic: 0 — Single statement of fact with proper attribution and no repetitive framing.
Malicious Account: 0 — Credible financial news source citing official Federal Reserve announcement.
Sensationalism: 0 — Neutral, technical language appropriate for economic reporting.
Naive Realism: 0 — Presents information objectively without claiming singular truth.

**Example 2:**
Article: WAKE UP PEOPLE! Everyone knows the government is controlling us through 5G towers! Thousands are saying it online. The mainstream media won't tell you the TRUTH because they're in on it!
Frequency Heuristic: 2 — Heavy appeals to popularity ('everyone knows,' 'thousands are saying') to establish credibility through repetition.
Malicious Account: 2 — Anonymous source spreading debunked conspiracy theories.
Sensationalism: 2 — All-caps words, exclamation marks, fear-mongering language designed to provoke emotional response.
Naive Realism: 2 — Presents conspiracy as absolute truth, dismisses all mainstream sources, and attempts to isolate readers.

**Example 3:**
Article: Is your food secretly KILLING you? Experts reveal the hidden dangers lurking in your kitchen. You won't believe what we found!
Frequency Heuristic: 1 — Uses populist framing ('hidden dangers') which suggests knowledge.
Malicious Account: 1 — Clickbait journalism from established outlet but not malicious.
Sensationalism: 2 — Clickbait with dramatic language and mystery ('you won't believe').
Naive Realism: 1 — Implies hidden truth without presenting the actual complexity of food safety.

**Example 4:**
Article: Tech company announces its AI will revolutionize healthcare and transform diagnosis forever. The CEO called it a 'game-changing breakthrough' during yesterday's product launch.
Frequency Heuristic: 1 — Repeats transformative framing.
Malicious Account: 0 — Corporate PR from legitimate company so promotional but not malicious.
Sensationalism: 2 — Extreme claims ('revolutionize,' 'transform forever,' 'game-changing').
Naive Realism: 1 — Presents corporate claims as reality without acknowledging uncertainty or limitations.

**Example 5:**
Article: According to multiple sources familiar with the matter, the investigation is ongoing. Officials declined to comment on specifics, citing the active nature of the case.
Frequency Heuristic: 0 — Reports lack of information honestly without speculation.
Malicious Account: 0 — Standard practice of protecting sources while noting limitations.
Sensationalism: 0 — Neutral tone and restrained language.
Naive Realism: 0 — Explicitly communicates uncertainty and ongoing nature of situation.

**Example 6:**
Article: 'Many people are saying the new policy is unfair. It's becoming clear that something needs to change. More and more citizens are questioning the decision every day.'
Frequency Heuristic: 2 — Repeating claims of rising support that lack clear attribution.
Malicious Account: 1 — Lacks verification.
Sensationalism: 1 — Emotionally charged framing ('unfair') and builds urgency but not extreme language.
Naive Realism: 1 — Implies growing consensus makes position correct without presenting other arguments.

**Example 7:**
Article: The new immigration bill sparked heated debate. Republican lawmakers argue it strengthens border security, while Democratic representatives contend it lacks humanitarian protections. Legal experts are divided on constitutional questions.
Frequency Heuristic: 0 — Multiple perspective stated once with attribution to distinct groups.
Malicious Account: 0 — Mainstream news outlet citing political figures and experts.
Sensationalism: 1 — Word 'heated' adds mild emotional tone but otherwise is balanced.
Naive Realism: 0 — Acknowledges multiple perspectives across parties and expert opinion.

**Example 8:**
Article: Shocking allegations have emerged against the senator. Anonymous sources describe a pattern of misconduct, though the senator's team calls the claims 'politically motivated lies' and notes no formal charges have been filed.
Frequency Heuristic: 1 — 'Pattern of misconduct' suggests multiple mentions.
Malicious Account: 1 — Anonymous sourcing.
Sensationalism: 2 — 'Shocking allegations' is designed to attract attention.
Naive Realism: 0 — Presents both accusation and denial. Also acknowledges lack of formal charges.

**Example 9:**
Article: As commonly understood by most experts in the field, renewable energy will replace fossil fuels within two decades. Some outlier researchers dispute this timeline, but the consensus is clear.
Frequency Heuristic: 1 — Frames majority view as established fact.
Malicious Account: 0 — Appears to be legitimate climate journalism citing expert community.
Sensationalism: 0 — Factual tone without emotional manipulation or exaggeration.
Naive Realism: 2 — Dismisses dissenting views and presents prediction as inevitable.

**Example 10:**
Article: Influencer @TrendyLifestyle123 claims this detox tea cured her chronic illness. The product has 50K followers and hundreds of testimonials, though medical professionals warn these supplements are unregulated.
Frequency Heuristic: 1 — Implies truth through follower count and testimonials, but includes medical warning.
Malicious Account: 2 — Suspicious account name and health claims.
Sensationalism: 1 — 'Cured chronic illness' is dramatic claim but is presented as a quote from influencer.
Naive Realism: 1 — Amplifies an unverified claim but includes expert skepticism.

### Article to Evaluate:
{article_text}


### Instructions:
1. Think step-by-step about the article’s tone, evidence, framing, and intent.
2. Identify linguistic cues that signal bias, repetition, exaggeration, or malicious intent.
3. Provide a numeric score and a justification for why that score was chosen.
4. Return **only** a valid JSON object containing the score and reasoning for each of the four factors.

### Output Format:
{{
    "frequency_heuristic": {{
        "score": 0|1|2,
        "reasoning": "Explanation"
  }},
    "malicious_account": {{
        "score": 0|1|2,
        "reasoning": "Explanation"
  }},
    "sensationalism": {{
        "score": 0|1|2,
        "reasoning": "Explanation"
  }},
    "naive_realism": {{
        "score": 0|1|2,
        "reasoning": "Explanation"
  }}
}}
"""

In [ ]:
generate(prompt_icl)

```json
{
    "frequency_heuristic": {
        "score": 1,
        "reasoning": "The article repeatedly frames Donald Trump's actions around the central narrative of him acting like a 'king' or an 'autocrat.' Phrases like 'unlocked power,' 'wannabe autocrat,' 'all-powerful,' 'pretensions to absolute power,' and 'monarchical moves' are used multiple times to reinforce this theme. It also references 'multiplying actions' and 'growing numbers of Americans' to suggest an increasing trend, implying truth through momentum rather than presenting isolated facts."
    },
    "malicious_account": {
        "score": 1,
        "reasoning": "The article is written from a clear anti-Trump perspective, which introduces a strong bias. While it attributes quotes to named individuals (e.g., Karoline Leavitt, Rand Paul, Colleen Connell), the overall framing and sourcing are selective to support its central thesis. The sourcing is not overtly malicious or fabricated, but it is one-sided. For example, the